In [1]:
import pandas as pd
import geopandas as gpd

/home/miglesia/anaconda2/envs/my_pymc_env/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/miglesia/anaconda2/envs/my_pymc_env/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
# Load natural gas locations data
df = pd.read_csv('./all_ng_data/all_data_757.csv', error_bad_lines = False, index_col=False)
df = df[['STATE', 'COUNTY', 'PLANT', 'ZIPCODE', 'STATUS', 'CAPACITY']]
df = df.loc[df.STATUS == 'Active']

b'Skipping line 7: expected 14 fields, saw 15\nSkipping line 9: expected 14 fields, saw 16\nSkipping line 10: expected 14 fields, saw 16\nSkipping line 11: expected 14 fields, saw 16\nSkipping line 12: expected 14 fields, saw 15\nSkipping line 13: expected 14 fields, saw 16\nSkipping line 17: expected 14 fields, saw 16\nSkipping line 18: expected 14 fields, saw 16\nSkipping line 29: expected 14 fields, saw 16\nSkipping line 30: expected 14 fields, saw 15\nSkipping line 35: expected 14 fields, saw 16\nSkipping line 36: expected 14 fields, saw 16\nSkipping line 37: expected 14 fields, saw 16\nSkipping line 41: expected 14 fields, saw 16\nSkipping line 42: expected 14 fields, saw 16\nSkipping line 44: expected 14 fields, saw 16\nSkipping line 50: expected 14 fields, saw 15\nSkipping line 51: expected 14 fields, saw 15\nSkipping line 55: expected 14 fields, saw 16\nSkipping line 57: expected 14 fields, saw 16\nSkipping line 62: expected 14 fields, saw 15\nSkipping line 63: expected 14 fiel

In [3]:
df.groupby(['STATE', 'COUNTY'])[['CAPACITY']].sum().sort_values(by = 'CAPACITY').tail()

,,CAPACITY
STATE,COUNTY,
CO,Weld,1724.0
TX,Colorado,2000.0
WV,Marshall,2140.0
MS,Jackson,2200.0
CO,Garfield,2350.0


### Zip codes

In [4]:
zip_df = gpd.read_file('./cb_2013_us_zcta510_500k/cb_2013_us_zcta510_500k.shp')

In [5]:
df_plant_poly = df.merge(zip_df[['ZCTA5CE10', 'geometry']], left_on = 'ZIPCODE', right_on = 'ZCTA5CE10', how = 'left')
df_plant_poly = df_plant_poly.dropna()

Create random points within zip code to simulate plant location. 

In [6]:
import random
from shapely.geometry import Polygon, Point

def get_random_point_in_polygon(poly):
    (minx, miny, maxx, maxy) = poly.bounds       
    while True:
        p = Point(random.uniform(minx, maxx), random.uniform(miny, maxy))
        if poly.contains(p):
            return p

df_plant_poly['point_1'] = [get_random_point_in_polygon(poly) for poly in df_plant_poly['geometry']]
df_plant_poly['point_2'] = [get_random_point_in_polygon(poly) for poly in df_plant_poly['geometry']]
df_plant_poly['point_3'] = [get_random_point_in_polygon(poly) for poly in df_plant_poly['geometry']]
df_plant_poly['point_4'] = [get_random_point_in_polygon(poly) for poly in df_plant_poly['geometry']]
df_plant_poly['point_5'] = [get_random_point_in_polygon(poly) for poly in df_plant_poly['geometry']]
df_plant_pts = df_plant_poly.drop(['ZCTA5CE10', 'geometry'], axis = 1)

#### Plants by county

In [7]:
# There are 572 plants in 201 counties
df_plant_pts.groupby(['STATE', 'COUNTY']).count().sort_index().head()

PLANT  ZIPCODE  STATUS  CAPACITY  point_1  point_2  point_3  \
STATE COUNTY                                                                  
AL    Conecuh       2        2       2         2        2        2        2   
      Conecuha      2        2       2         2        2        2        2   
      Escambia      2        2       2         2        2        2        2   
      Mobile        6        6       6         6        6        6        6   
      Monroe        6        6       6         6        6        6        6   

                point_4  point_5  
STATE COUNTY                      
AL    Conecuh         2        2  
      Conecuha        2        2  
      Escambia        2        2  
      Mobile          6        6  
      Monroe          6        6

In [20]:
### Save 
df_plant_pts['geometry'] = df_plant_pts['point_2']
gpd.GeoDataFrame(df_plant_pts.drop([col for col in df_plant_pts.columns if 'point' in col], axis = 1)
                ).to_file("NG_point_2.shp")

#### Size distribution (log)

In [101]:
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
np.log10(df_plant_pts.CAPACITY).hist(bins = 20)
plt.show()

### Mapping

In [1]:
import os

def save_geojson(gdf, filename = 'test.geojson'):
    try:    # Delete if geojson exists, cause overwriting is not supported
        os.remove('./geojson/'+filename)
    except OSError:
        pass
    
    gdf.to_file('./geojson/'+filename, driver="GeoJSON", encoding='utf-8')
    
from mapbox import Uploader
import json

def upload_file(data, name, username = 'matuteiglesias', token = 'sk.eyJ1IjoibWF0dXRlaWdsZXNpYXMiLCJhIjoiY2puODA4bW8xMGV1dzNrcGtiOGp6NXQ5aCJ9.DohKmjn_o6MK1Y4Q5FG8ew'):
    try:    # Clear upload file if exists
        os.remove('./upload_data.geojson')
    except OSError:
        pass
    
    # Dump into file for upload    
    with open('./upload_data.geojson', 'w') as outfile:
        json.dump(data, outfile)

    service = Uploader(access_token=token)
    with open('./upload_data.geojson', 'rb') as src:
        # Acquisition of credentials, staging of data, and upload
        # finalization is done by a single method in the Python SDK.
        upload_resp = service.upload(src, username+'.'+name)
        

In [4]:
import geopandas as gpd
# for i in ['1', '2']:
#     gdf = gpd.GeoDataFrame(df_plant_pts[['STATE','COUNTY','PLANT','ZIPCODE','STATUS','CAPACITY', 'point_'+i]].rename(columns = {'point_'+i: 'geometry'}))
#     name = 'NL_plants_'+i
#     save_geojson(gdf, name+'.geojson')
    
# Load Oil Refineries Location data
gdf = gpd.read_file('./locations_data/U.S._Petroleum_Refineries.shp')
name = 'Oil_Refineries'
save_geojson(gdf, name+'.geojson')

username = 'matuteiglesias'
token = 'sk.eyJ1IjoibWF0dXRlaWdsZXNpYXMiLCJhIjoiY2puODA4bW8xMGV1dzNrcGtiOGp6NXQ5aCJ9.DohKmjn_o6MK1Y4Q5FG8ew'

files = os.listdir('./geojson/')
names = [name.split('.')[0] for name in files]

for i in range(len(files)):
    print(names[i])
    data = json.load(open('./geojson/'+files[i]))

    try:
        upload_file(data, names[i])
    except:
        pass

Oil_Refineries
